# 1. Formulação do problema

    Variáveis de Decisão:
        ti​ - Tempo de pouso do avião i (variável contínua);
        xij​ - Variável binária que vale 1 se o avião i pousar antes do avião j, 0 caso contrário;
        ei - Adiantamento do tempo ideal de pouso para o avião i (ei ≥ 0);
        di - Atraso do tempo ideal de pouso para o avião i (di ≥ 0).

    Parâmetros:
    
        Ri - tempo de sua detecção pelo radar;
        Ei - tempo inicial de pouso;
        Ti - tempo ideal para o pouso;
        Li - tempo final que o avião i pode pousar;
        
    Função Objetivo:

        Minimizar a penalidade total de pousos fora do tempo ideal:

## **Minimizar ∑ (gi * ei + hi * di)**

Onde:

    gi​ e hi​ são as penalidades por pousar antes ou depois do tempo ideal, respectivamente.

    S.a:
        
        Restrições de Viabilidade:
        
            Cada avião deve pousar dentro do seu intervalo permitido:

                Ei ​≤ ti ​≤ Li ​∀i 

        Restrições de Separação:
            O intervalo entre os pousos deve ser suficiente para garantir segurança:

                tj ≥ ti + sij - M(1 - xij) ∀i,j, i≠j

                M representa um valor grande.
        
        Variáveis Binárias:

                xij​ + xji ​= 1 ∀i, j, i≠j

        Cálculo de adiantamento e atraso:
                ei ​≥ Ti ​− ti  ​∀i
                di ≥ ti − Ti  ∀i
                



# Resolvendo usando o solver GLPK

In [ ]:
from pulp import LpProblem, LpVariable, LpMinimize, lpSum, LpBinary, LpStatus

def solverSequenciamento(n, tempos, separacoes, penalidades):
    E = [temp[0] for temp in tempos]
    T = [temp[1] for temp in tempos]
    L = [temp[2] for temp in tempos]
    g = [penalidade[0] for penalidade in penalidades]
    h = [penalidade[1] for penalidade in penalidades]

    # Criar modelo
    modelo = LpProblem("SequenciamentoPousos", LpMinimize)

    # Variáveis de decisão
    t = [LpVariable(f"t_{i}", lowBound=E[i], upBound=L[i], cat='Continuous') for i in range(n)]
    e = [LpVariable(f"e_{i}", lowBound=0, cat='Continuous') for i in range(n)]
    d = [LpVariable(f"d_{i}", lowBound=0, cat='Continuous') for i in range(n)]
    x = [[LpVariable(f"x_{i}_{j}", cat=LpBinary) if i != j else None for j in range(n)] for i in range(n)]

    # Função objetivo: minimizar penalidades
    modelo += lpSum(g[i] * e[i] + h[i] * d[i] for i in range(n)), "PenalidadeTotal"

    # Restrições de separação e sequenciamento
    M = max(L) + max(separacoes[i][j] for i in range(n) for j in range(n) if i != j)  # Valor grande

    for i in range(n):
        for j in range(n):
            if i != j:
                modelo += t[j] >= t[i] + separacoes[i][j] - M * (1 - x[i][j]), f"Separacao_{i}_{j}"
                modelo += x[i][j] + x[j][i] == 1, f"Sequencia_{i}_{j}"

    # Cálculo de adiantamento e atraso
    for i in range(n):
        modelo += e[i] >= T[i] - t[i], f"Adiantamento_{i}"
        modelo += d[i] >= t[i] - T[i], f"Atraso_{i}"

    # Resolver o problema
    modelo.solve()

    # Extrair solução
    if LpStatus[modelo.status] == "Optimal":
        temposPouso = [t[i].varValue for i in range(n)]
        return temposPouso
    else:
        print("Não há solução ótima.")
        return None


#Acessar o site com as instâncias
# Exemplo de chamada da função
#n = 3  # Número de aviões

#tempos = [(5, 10, 20), (8, 15, 25), (12, 18, 30)]  # E, T, L para cada avião
#separacoes = [[0, 3, 2], [3, 0, 4], [2, 4, 0]]  # Matriz de separações
#gPenalidades = [(2, 3), (1, 4), (3, 2)]  # Penalidades (g, h) para cada avião

#solucao = solverSequenciamento(n, tempos, separacoes, gPenalidades)
#if solucao:
#    print("Sequência ótima de pousos:", solucao)


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/guilherme/miniconda3/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/7ea749aecf2d4b0a824aa88c8908ecf5-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/7ea749aecf2d4b0a824aa88c8908ecf5-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 23 COLUMNS
At line 84 RHS
At line 103 BOUNDS
At line 116 ENDATA
Problem MODEL has 18 rows, 15 columns and 42 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 2 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0004I processed model has 12 rows, 12 columns (3 integer (3 of which binary)) and 32 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 2 integers unsatisfied sum - 